# Pyladies meetup <img src="https://raw.githubusercontent.com/pyladies/pyladies-assets/master/geek/png/pylady_geek_partial.png" style="display:inline" width="150"  align="right">
# 4. házi feladathoz tartozó szorgalmi feladat

A hivatalnok látja, hogy Geeky épp a kimutatások részleteit böngészi, így hozzáteszi:
- ezek a legsürgetőbb információk amikre szükségünk van, a többivel meg tudjuk várni az elemzőnket. Nagyon könnyű dolga lesz azok után, hogy ilyen szépen előkészítette az adatokat. De ha van kedve, akkor egy jegeskávé kuponért (plusz pontért) cserébe lepjen meg minket, hátha eszébe jut valami olyan nézet, ami nekünk nem, és ezzel feldobhatjuk a minisztériumnak adott jelentésünket.


In [ ]:
import pandas as pd
df = pd.read_csv('bergengoc_nepszamlalas_tisztitott.csv', sep = ',')
df

## Keresztnevek elemzése
- Melyik a leggyakoribb női keresztnév?
- Melyik a leggyakoribb férfi keresztnév?
- Melyik népcsoportban mi a leggyakoribb név?
- Melyik megyében mi a leggyakoribb név?

     ### Eredmény:
     Annyi érdekességre fény derült, hogy a női/férfi névadási szokásaik merőben eltérőek a nálunk megszokottól :)

In [ ]:
# Leggyakoribb keresztnevek nemenként
nevek_nemenkent = df.groupby('nem')['keresztnev'].value_counts()
noi_leggyakoribb = nevek_nemenkent['no'].idxmax()
ferfi_leggyakoribb = nevek_nemenkent['ferfi'].idxmax()

# Leggyakoribb keresztnevek nepcsoportonként
nevek_nepcsoportonkent = df.groupby('nepcsoport')['keresztnev'].value_counts()

# Leggyakoribb keresztnevek megyénként
nevek_megyenkent = df.groupby('lakhely_megye')['keresztnev'].value_counts()

# Az eredmények kiírása
print(f"Leggyakoribb női név: {noi_leggyakoribb}")
print(f"Leggyakoribb férfi név: {ferfi_leggyakoribb}")
print('----------------------------------------------')

for nepcsoport in df['nepcsoport'].unique():
    print(f"{nepcsoport} népcsoportban a leggyakoribb név: {nevek_nepcsoportonkent[nepcsoport].idxmax()}")

print('----------------------------------------------')

for megye in df['lakhely_megye'].unique():
    print(f"{megye} megyében a leggyakoribb név: {nevek_megyenkent[megye].idxmax()}")


In [ ]:
# Érdekes, hogy "Alexander" lett a leggyakoribb női név. Kicsit tanulmányozom még a női/férfi neveket...

noi_nevek_lista = df[df.nem == 'no'].sort_values(by='keresztnev')['keresztnev'].unique().tolist()
print(', '.join(noi_nevek_lista))

print('-----------------------------------------------------------')

ferfi_nevek_lista = df[df.nem == 'ferfi'].sort_values(by='keresztnev')['keresztnev'].unique().tolist()
print(', '.join(ferfi_nevek_lista))

### Korcsoportok elemzése
- Soroljuk be a lakosokat korcsoportokba, hogy lássuk hogy alakul a fiatalok / idősek aránya
- Vizsgáljuk meg a korcsoportok eloszlását az egyes népcsoportokban
- Vizsgáljuk meg a korcsoportok eloszlását az egyes megyékben
- Vizsgáljuk meg a nemek eloszlását az egyes korcsoportokon belül
- Határozzuk meg az átlagos BMI és magasság értéket korcsoportonként

    ### Eredmény
    Annyi érdekességre derült fény, hogy a bergengóc babák már eleve felnőtt méretekkel (magasság és BMI) születnek :)

In [ ]:
# Korcsoportok definiálása
bins = [0, 20, 40, 60, 80, 110, 150]
labels = ['0-20', '20-40', '40-60', '60-80', '80-110', '110+']

df['korcsoport'] = pd.cut(df['kor'], bins=bins, labels=labels, right=False)

# Korcsoportok eloszlásának vizsgálata
df['korcsoport'].value_counts().sort_index()


In [ ]:
# A korcsoportok eloszlása az egyes népcsoportokban
korcsoport_eloszlas_nepcsoportonkent = df.groupby('nepcsoport')['korcsoport'].value_counts().unstack().fillna(0)

# A korcsoportok eloszlása az egyes megyékben
korcsoport_eloszlas_megyenkent = df.groupby('lakhely_megye')['korcsoport'].value_counts().unstack().fillna(0)

# A nemek eloszlása az egyes korcsoportokban
nemek_eloszlas_korcsoportonkent = df.groupby('korcsoport')['nem'].value_counts().unstack().fillna(0)

print(korcsoport_eloszlas_nepcsoportonkent)
print('-------------------------------------------------------------')
print(korcsoport_eloszlas_megyenkent)
print('-------------------------------------------------------------')
print(nemek_eloszlas_korcsoportonkent)


In [ ]:
average_values = df.groupby('korcsoport')[['BMI', 'magassag']].mean()
std_values = df.groupby('korcsoport')[['BMI', 'magassag']].std()

result = pd.concat([average_values, std_values], axis=1, keys=['Átlag', 'Szórás'])

result.columns = ['_'.join(col).strip() for col in result.columns.values]
result = result.reindex(columns=['Átlag_magassag', 'Szórás_magassag', 'Átlag_BMI', 'Szórás_BMI'])
result

In [ ]:
# Érdekes, hogy a 0-20 korcsoportban is ugyanúgy alakul az átlag magasság (és szórás), 
# mint a többiben. Lekérem ehhez a korcsoporthoz tartozó 'kor' és 'magasság' értékeket, 
# hogy egy kicsit több infok legyen erről...

df[df.korcsoport == '0-20'][['kor', 'magassag', 'BMI']].sort_values(by='kor')

# Hát, nem lennék azon bergengóc anyák helyében, akiknek 211 cm-es, közel 170 kg-os babát kell szülniük :))
# Bár, ki tudja, náluk hogy megy ez a szülés/születés kérdés... 
# Remélem, ők csak teremtődnek (mondjuk egy random generátor segítségével:)) ekkora "babának"...

In [ ]:
# A 211 cm-es, 38.07-es BMI-s bergengóc baba súlyáak meghatározása - just out of curiosity

def suly_kalkulator(magassag, BMI):
    # Ellenőrzi, hogy a bemeneti értékek számok-e
    if ((type(magassag) not in [int, float]) or (type(BMI) not in [int, float])):
        raise ValueError("A magasság és a BMI értékeknek számoknak kell lenniük!")

    # Ellenőrzi a magasság értékét
    if magassag <= 70 or magassag > 250:
        raise ValueError("A magasság értékének 70 cm és 250 cm között kell lennie!")

    # Ellenőrzi a BMI értékét
    if BMI <= 10 or BMI > 50:
        raise ValueError("A BMI értékének 10 és 50 között kell lennie!")
        
    # Ha minden rendben érték rendben van, akkor elvégzi a számítást
    suly = BMI * (magassag / 100)**2
    suly_kerekitve = round(suly, 2)
    
    #Végül kiírja az eredményt
    print(f"A megadott adatok alapján a személy súlya {suly_kerekitve} kg")

suly_kalkulator(211, 38.07)

### Korrelációs elemzések
- Vizsgáljuk meg, van-e valamilyen összefüggés a numerikus változók között?
- Vizsgáljuk meg, van-e valamilyen összefüggés a kategórikus változók között?

    ### Eredmény
    Sem a numerikus, sem a kategórikus változók között nem fedezhető fel szignifikáns kapcsolat

In [ ]:
# Numerikus változók közti korreláció
df[['kor', 'magassag', 'BMI']].corr()


In [ ]:
# Kategórikus változók közti korreláció
import researchpy as rp

kategorikus_valtozok = ['korcsoport', 'nem', 'nepcsoport', 'lakhely_megye']

# Az összes lehetséges változópáros korrelációjának kiszámítása
for i in range(len(kategorikus_valtozok)):
    for j in range(i+1, len(kategorikus_valtozok)):
        table, results = rp.crosstab(df[kategorikus_valtozok[i]], df[kategorikus_valtozok[j]], test= 'chi-square')
        
        chi_square = results.iloc[0, 1]
        p_value = results.iloc[1, 1]
        cramers_v = results.iloc[2, 1]
        
        print(f"Párosítás: {kategorikus_valtozok[i]} és {kategorikus_valtozok[j]}")
        print(f"Pearson Chi-square érték: {chi_square:.4f}")
        print(f"p-value: {p_value:.4f}")
        print(f"Cramér V: {cramers_v:.4f}")
        print("-" * 50)  

### Pivot-table kimutatások
- Készítsünk Pivot-table formátumú kimutatásokat a numerikus változók átlagos értékéről a kategórikus változók összes lehetséges kombinációjára!

    ### Eredmény
    Semmi kiugró adatot nem látok, ami további elemzésre adna okot

In [ ]:
kategorikus_valtozok = ['nem', 'nepcsoport', 'lakhely_megye']
numerikus_valtozok = ['kor', 'magassag', 'BMI']

# Az összes lehetséges változópárosra készítünk pivot-táblát:
for i in range(len(kategorikus_valtozok)):
    for j in range(i+1, len(kategorikus_valtozok)):
        print(f"Pivot-table {kategorikus_valtozok[i].upper()} és {kategorikus_valtozok[j].upper()} alapján:")
        for num_var in numerikus_valtozok:
            pivot = df.pivot_table(values=num_var, 
                                   index=kategorikus_valtozok[i], 
                                   columns=kategorikus_valtozok[j], 
                                   aggfunc='mean')
            print(f"ÁTLAG {num_var.upper()}:\n", pivot, "\n")
        print("-" * 50)